# Dataset YouTube API

In [1]:
import googleapiclient.discovery
from resources.keys import api_key as DEVELOPER_KEY
import pandas as pd
from resources.basicIO import InputOutput as IO

## API Call

In [2]:
# API CALL
api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

## Fetch Channels from YouTube

In [3]:
queries = ["lakhimpur violence", "farmer protest","kisan andolan","farmer bill", 
           "agriculture reform bill 2020","kisan morcha","rakesh tikait",
           "kisan mahapanchayat","farmers march","Farmer Protest LatestNews",
           "Doubling farmers income","krishi mandi","farmlaws","Krishi Bill 2020",
           "Haryana farmers Protest","farmer protest delhi border","A Day at the Tikri Border",
           "farmers protest in Tohana","tractor rally delhi","kisan vs police","kisan riot",
           "kisan taliban","farmers bill withdrawn","kisan bill","farmer bill latest news",
           "minimum support price farm bill","farm law repealed"]
    

# lists
video_Id = []
video_Title = []
channel_Id = []
channel_name = []
video_Date = []

# take multiple search results
for i in range(len(queries)):
    request = youtube.search().list(
        part="snippet",
        maxResults=50,
        q=queries[i],
    )
    response = request.execute()
    videos_total = len(response['items'])

    # all parsed videos
    for i in range(videos_total):
        # insert if unique video id

        try:
            if response['items'][i]['id']['videoId'] not in video_Id:
                video_Id.append(response['items'][i]['id']['videoId'])
                video_Title.append(response['items'][i]['snippet']['title'])
                channel_Id.append(response['items'][i]['snippet']['channelId'])
                channel_name.append(
                    response['items'][i]['snippet']['channelTitle'])
                video_Date.append(
                    response['items'][i]['snippet']['publishedAt'])
        except:
            print('VideoId miss')

VideoId miss


### Make a csv Database for search results

In [4]:
pd.DataFrame({'videoId': video_Id,
              'title': video_Title,
              'channelId': channel_Id,
              'channelName': channel_name,
              'date': video_Date}).to_csv('datasets/search_results.csv', index=False)

In [5]:
# display the dataset
output = pd.read_csv("datasets/search_results.csv")
display(output)


,videoId,title,channelId,channelName,date
0,l1oobMY7CE4,"Lakhimpur Kheri Violence - 8 people killed, Ca...",UCrC8mOqJQpoB7NuIMKIS6rQ,Study IQ education,2021-10-04T05:45:19Z
1,Bvcd0LK2K88,Lakhimpur Kheri Violence: 7 Shocking Videos De...,UCYPvAwZP8pZhSMW8qs7cVCw,India Today,2021-10-05T10:27:54Z
2,v_xiGjSd6aw,Watch: Full 1 minute 45 second Video of Lakhim...,UCwqusr8YDwM-3mEYTDeJHzw,Republic World,2021-10-05T09:05:09Z
3,Gz7kaVLbvJY,DNA: लखीमपुर खीरी हिंसा में हुई मौतों का जिम्म...,UCIvaYmXn910QMdemBG3v1pQ,Zee News,2021-10-04T16:31:47Z
4,XcMqzv3KWNA,Lakhimpur Kheri Violence: Video Of Questioning...,UCZFMm1mMw0F81Z37aaEzTUA,NDTV,2021-10-06T05:52:52Z
...,...,...,...,...,...
1072,fi8IjCOFMRA,Farm Laws Repealed : जब PM Modi ने मांगी क्षमा...,UClmlqOOktUTpfW0EkTiqSjQ,Aaj Tak HD,2021-11-19T04:56:05Z
1073,j05yeuni4SM,Farm Laws Repealed | ‘We Will Wait Until Farm ...,UCSaf-7p3J_N-02p7jHzm5tA,The Quint,2021-11-19T12:01:49Z
1074,RAahNBM3D68,Farm Laws repeal | Punjab Farmers expresses ha...,UCRWFSbif-RFENbBrSiez1DA,ABP NEWS,2021-11-19T09:34:59Z
1075,hOyXuuWGFvw,DNA: कानून रद्द होने से देश के बाकी किसानों का...,UCIvaYmXn910QMdemBG3v1pQ,Zee News,2021-11-19T20:24:43Z


## Fetch Comments from YouTube

In [6]:
search_results = pd.read_csv("datasets/search_results.csv")

# lists
user_name = []
user_comment = []
user_likes_count = []
video_id = []

for i in range(len(search_results)):
    try:
        # ith video
        videoID = (search_results['videoId'][i])

        # next pagen token
        npt = None
        pages = 30

        for j in range(pages):

            # if next page does not exist
            if(npt == None):
                request = youtube.commentThreads().list(
                part="snippet,replies",
                maxResults=100,
                order="relevance",
                videoId=videoID,
                )
            else:
                request = youtube.commentThreads().list(
                    part="snippet,replies",
                    maxResults=100,
                    order="relevance",
                    videoId=videoID,
                    pageToken=npt
                )
            
            response = request.execute()

            if int(response['pageInfo']['totalResults']) == 0:
                break
            if "nextPageToken" in response:
                npt = response['nextPageToken']
            else:
                npt = None

            comments_total = len(response['items'])

            for j in range(comments_total):
                video_id.append(videoID)
                user_comment.append(
                    response['items'][j]['snippet']['topLevelComment']['snippet']['textOriginal'])
                user_name.append(
                    response['items'][j]['snippet']['topLevelComment']['snippet']['authorDisplayName'])
                user_likes_count.append(
                    response['items'][j]['snippet']['topLevelComment']['snippet']['likeCount'])

    except:
        #print('Comment disabled for ',search_results['videoId'][i])
        pass

### Make a csv for comments

In [7]:
# save into csv
pd.DataFrame({'Username': user_name, 'Comment': user_comment, 'Likes': user_likes_count,
             'Video ID': video_id}).to_csv('datasets/comments.csv', index=False)

In [8]:
# display the dataset
output = pd.read_csv("datasets/comments.csv")
display(output)

,Username,Comment,Likes,Video ID
0,Study IQ education,Click here to buy full UPSC course https://pla...,37,l1oobMY7CE4
1,Vishal Choudhury,Not a single day without the UP news.,549,l1oobMY7CE4
2,Arun Kumar Sahoo,Think about their families... What they must b...,134,l1oobMY7CE4
3,Abodh S,"love your content, as it is totally unbiased :)",89,l1oobMY7CE4
4,Nita Agarwal,Jaise-2 elections paas aayenge ye sab badhega ...,805,l1oobMY7CE4
...,...,...,...,...
412440,Vish,Pagal sardar,0,vczmNojMXm8
412441,Manpreet Singh,Kisan majdoor ekta zindabaad,0,vczmNojMXm8
412442,Vish,Pagal sardar,0,vczmNojMXm8
412443,Manpreet Singh,Kisan majdoor ekta zindabaad,0,vczmNojMXm8


### Save comments in a text file

In [9]:
corpus = list(set(user_comment))
IO.save_text('datasets/comments.txt',corpus)